In [1]:
import os
import numpy as np
import librosa
from utils.dsp import *
import models.vqvae as vqvae
from utils import env
import config
import torch
import resemblyzer

In [67]:
model = vqvae.Model(rnn_dims=896, fc_dims=896, global_decoder_cond_dims=resemblyzer.hparams.model_embedding_size,
                    upsample_factors=(4, 4, 4), normalize_vq=True, noise_x=True, noise_y=True).cuda()


pad += 1 * 2 * 1
pad += 2 * 2 * 1
pad += 4 * 2 * 1
pad += 8 * 3 * 1
pad += 8 * 2 * 1
pad += 16 * 3 * 1
pad += 16 * 2 * 1
pad += 32 * 3 * 1
pad += 32 * 2 * 1
pad += 64 * 3 * 1
Trainable Parameters: 17.303 million


In [68]:
import importlib
importlib.reload(env)
model_type = 'vqvae'
model_name = f'{model_type}.43.upconv'
data_path = config.multi_speaker_data_path
paths = env.Paths(model_name, data_path)
prev_path = paths.model_path()
step = env.restore(prev_path, model)

In [69]:
def process_file(path):
    floats = load_wav(path, encode=False)
    quant = (floats * (2**15 - 0.5) - 0.5).astype(np.int16)
    if max(abs(quant)) < 2048:
        print(f'audio fragment too quiet ({max(abs(quant))}), skipping: {path}')
        return
    if len(quant) < 10000:
        print(f'audio fragment too short ({len(quant)} samples), skipping: {path}')
        return
    return quant

In [70]:
my_sound = process_file("test_audio/my.wav")

In [71]:
def audio_torch(wav):
    gt = (wav.astype(np.float32) + 0.5) / (2**15 - 0.5)
    extended = np.concatenate([np.zeros(model.pad_left_encoder(), dtype=np.float32), gt, np.zeros(model.pad_right(), dtype=np.float32)])
    extended = extended[np.newaxis, ...]
    return torch.FloatTensor(extended).cuda()
me_torch = audio_torch(my_sound)
me_torch.shape

torch.Size([1, 61252])

In [72]:
import pickle

data_path = config.multi_speaker_data_path
with open(f'{data_path}/index.pkl', 'rb') as f:
    index = pickle.load(f)
dataset = env.MultispeakerDataset(index, data_path)

In [73]:
generated = model.forward_generate(torch.FloatTensor(dataset[0][0][np.newaxis, ...]).cuda(), me_torch)

entropy: 1.896382451057434
55201/55296 -- Speed: 1293 samples/sec

In [74]:
from IPython.display import Audio as aud
gen = generated[0]
gen = gen.cpu().numpy()
aud(gen, rate=22050)